In [ ]:
# Cell to copy data from Google Storage to the runtime

import os

if not os.path.isdir("/content/Separate Corpuses/FullCorpus/") or not os.path.isdir("/joey_experiments/"):
    !gsutil -m cp -r "gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/" .
    !gsutil -m cp -r "gs://mytranslator-298419-vcm/joey_experiments/" .

Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tc.model...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tok.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tok.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Bel.txt_tok.txt_true.txt...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tc.model...
Copying gs://mytranslator-298419-vcm/RuBeCorpus_BelapanNews/Separate Corpuses/FullCorpus/Corpus_Rus.txt_tok.txt_true.txt...
| [8/8 files][651.5 MiB/651.5 MiB] 100% Done                                

In [ ]:
# Helper settings

source_language = "ru"
target_language = "be"
lang_pair = source_language+target_language
tag = "new_full"
model_type = "bpe"

use_cuda = True

if model_type == "bpe":
    # Learn BPEs on the training data.
    number_of_splits = 5000
    os.environ["data_path"] = os.path.join("joeynmt", "data", source_language + target_language)
    bpe_codes_file = "bpe.codes."+str(number_of_splits)

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["lang_pair"] = lang_pair
os.environ["tag"] = tag
os.environ["model_type"] = model_type

runtime_path = "/content/joey_experiments/%s-%s-%s" % (source_language, target_language, tag)
os.environ["runtime_path"] = runtime_path
!echo $runtime_path

raw_source_file = r"/content/FullCorpus/Corpus_Rus.txt"
raw_target_file = r"/content/FullCorpus/Corpus_Bel.txt"

source_tc_model = f"{raw_source_file}_tc.model"

# They should both have the same length.
! wc -l "$raw_source_file"
! wc -l "$raw_target_file"

/content/joey_experiments/ru-be-new_full
429479 /content/FullCorpus/Corpus_Rus.txt
429479 /content/FullCorpus/Corpus_Bel.txt


In [ ]:
%%capture
# installing sacremoses - tokenization library
! pip install sacremoses

# Install JoeyNMT
! git clone --depth 1 --branch 1.0 https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

In [ ]:
# testing if joeynmt installed correctly
! cd joeynmt; python3 -m unittest

................................s.......................
----------------------------------------------------------------------
Ran 56 tests in 1.264s

OK (skipped=1)


In [ ]:
%%capture
# installing cuda
if use_cuda:
    !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
    !sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
    !sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
    !sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"
    !sudo apt-get update
    !sudo apt-get -y install cuda

In [ ]:
# copying data from runtime folder to joeynmt
!mkdir -p joeynmt/models/${src}${tgt}_transformer_new/ && cp -r "$runtime_path"/models/{config.yaml,config_non_cuda.yaml} joeynmt/models/${src}${tgt}_transformer_new/
!mkdir -p joeynmt/data/${src}${tgt}/ && cp -r "$runtime_path/data/"* joeynmt/data/${src}${tgt}/
!cp -r joeynmt/data/${src}${tgt}/{bpe.codes.5000,vocab.ru} ./

In [ ]:
# Functions to test translation quality in two modes: 'interactive' and 'file'.

from sacrebleu import corpus_bleu

def interactive_translate(text):
    os.environ["config"] = "config.yaml" if use_cuda else "config_non_cuda.yaml"
    ! echo "$text" | sacremoses tokenize | sacremoses truecase -m "$source_tc_model" | subword-nmt apply-bpe -c "$bpe_codes_file" --vocabulary vocab.$source_language > "joeynmt/in.txt"
    ! cd joeynmt; python3 -m joeynmt translate "$runtime_path/models/$config" < in.txt 2> /dev/null | sacremoses detruecase 2> /dev/null | sacremoses detokenize 2> /dev/null | sed "s/ '/'/" | sed "s/' /'/"


# use_cuda = true is strongly recommended for file translation!
def file_translate(file):
    os.environ["config"] = "config.yaml" if use_cuda else "config_non_cuda.yaml"
    ! sed -i 's/"//g' "$file"
    ! sacremoses tokenize < "$file" | sacremoses truecase -m "$source_tc_model" | subword-nmt apply-bpe -c "$bpe_codes_file" --vocabulary vocab.$source_language > "joeynmt/in.txt"
    ! cd joeynmt; python3 -m joeynmt translate "$runtime_path/models/$config" < in.txt 2> /dev/null | sacremoses detruecase 2> /dev/null | sacremoses detokenize 2> /dev/null | sed "s/ '/'/" | sed "s/' /'/" > "joey_pred.txt"
    outfile = f"{file}_pred"
    os.rename(os.path.join('joeynmt', 'joey_pred.txt'), outfile)
    return outfile


def get_bleu(reference_file, prediction_file):
    with open(reference_file, 'r', encoding="utf-8") as reference_f, \
            open(prediction_file, 'r', encoding="utf-8") as prediction_f:
        real = reference_f.readlines()
        prediction = prediction_f.readlines()

    return corpus_bleu(prediction, [real]).score

In [ ]:
#Running tests for base translation model and two domain adaptations

path_to_test_data_folder = '/content/joey_experiments/ru-be-new_full/data_test'

for mode in ['base', 'medical', 'legal']:
    print(f"MODE: {mode}.")
    os.environ["checkpoint"] = mode + "_best.ckpt"
    ! cp -r "$runtime_path"/models/"$checkpoint" joeynmt/models/${src}${tgt}_transformer_new/

    interactive_translate("Как объяснили медработники, сделано для того, чтобы защитить стены от возможных ударов каталок.")
    interactive_translate("Коллективный договор может иметь приложения, являющиеся его неотъемлемой составной частью.")
    interactive_translate("В теплое время года на прилегающей к стационарным объектам территории была организована работа летних кафе.")

    for test_file_prefix in ['med_sentences', 'euroradio', 'belapan', 'legal']:
        in_file = os.path.join(path_to_test_data_folder, f"{test_file_prefix}_ru.txt")
        pred_file = file_translate(in_file)
        real_file = os.path.join(path_to_test_data_folder, f"{test_file_prefix}_be.txt")
        print(f"{test_file_prefix}: {get_bleu(real_file, pred_file)}.")
    
    ! rm joeynmt/models/${src}${tgt}_transformer_new/"$checkpoint"

MODE: base.
Як растлумачылі медработнікі, зроблена для таго, каб абараніць сцены ад магчымых удараў каталок.
Калектыўная дагавор можа мець прыкладанні, якія з'яўляюцца яго неад'емнай складнай часткай.
У цёплае час года на прылеглай да стацыянарных аб'ектаў тэрыторыі была арганізавана праца летніх кафэ.
med_sentences: 70.01743131026568.
euroradio: 69.8678115859498.
belapan: 75.31419717393905.
legal: 73.64540803530869.
MODE: medical.
Як патлумачылі медработнікі, зроблена для таго, каб абараніць сцены ад магчымых удараў каталок.
Калектыўны дамову можа мець прыкладання, якія з'яўляюцца яго неад'емнай складнай часткай.
У цеплае час года на прылеглай да стацыянарных аб'ектаў тэрыторыі была арганізавана праца лётных кафе.
med_sentences: 83.06449670035552.
euroradio: 58.06088338423993.
belapan: 58.8655141280291.
legal: 60.92894169379268.
MODE: legal.
Як тлумачылі адработнікі, зроблена для таго, каб абароныць стены ад магчымых удараў каталок
Калектыўны дагавор можа мець прыкладання, якія з'яўля